# 모델 앙상블

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input, Reshape

import time

# 회귀 문제

In [ ]:
x = np.arange(-1, 1, 0.01)
np.random.shuffle(x)
y = np.sin(x)

i = int(x.shape[0]*0.8)

train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]


In [ ]:
def build_model():  
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")

  return model

In [ ]:
def build_model_A():  
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")

  return model

def build_model_B():  
    model = keras.Sequential()
    model.add(Input((1,)))
    model.add(Dense(10, activation='tanh'))
    model.add(Dense(10, activation='tanh'))
    model.add(Dense(1))

    model.compile(optimizer="Adam", loss="mse")

    return model

def build_model_C():  
    model = keras.Sequential()
    model.add(Input((1,)))
    model.add(Dense(10, activation='sigmoid'))
    model.add(Dense(10, activation='sigmoid'))
    model.add(Dense(1))

    model.compile(optimizer="Adam", loss="mse")

    return model

In [ ]:
n_members = 3

In [ ]:
models = []
models.append(build_model_A())
models.append(build_model_B())
models.append(build_model_C())
# for _ in range(n_members):
	# model = build_model()
	# models.append(model)

In [ ]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32)

In [ ]:
y_ = np.array([model.predict(test_x) for model in models])

In [ ]:
# 아래 3줄과 같은 코드
y_ = [model.predict(test_x) for model in models]
# y_ = []
# for model in models:
#     y_.append(model.predict(test_x))

y_ = np.array(y_)
print("y_.shape =", y_.shape)
outcomes = np.mean(y_,axis=0).squeeze()
print("outcomes.shape =", outcomes.shape)

mse = tf.keras.losses.mean_squared_error(test_y, outcomes).numpy()
print("mse =", mse)

# 분류 문제

In [ ]:
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.mnist.load_data()

train_x = raw_train_x / 255.
test_x = raw_test_x / 255.

train_y = raw_train_y
test_y = raw_test_y

In [ ]:
def build_model():  
  model = keras.Sequential()
  model.add(Input((28,28)))
  model.add(Reshape((28,28,1)))
  model.add(Conv2D(32, (3, 3), padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:

n_members = 3

In [ ]:
models = []
for _ in range(n_members):
	model = build_model()
	models.append(model)

In [ ]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32)

In [ ]:
from sklearn.metrics import accuracy_score

y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_)
summed = np.sum(y_, axis=0)
print(summed[0])
outcomes = np.argmax(summed, axis=1)
print(outcomes)

acc = accuracy_score(test_y, outcomes)
print(acc)


# 2진 분류 문제

In [ ]:
!wget https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/data/sonar.csv

In [ ]:
raw_data = pd.read_csv("sonar.csv")
data = raw_data.to_numpy()
np.random.shuffle(data)

x = data[:,:60]
y = data[:,60]

i = int(len(x)*0.8)
train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]

In [ ]:
def build_model():  

  model = keras.Sequential()
  model.add(Dense(10, activation='relu', input_shape=(60,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["acc"])

  return model

In [ ]:
n_members = 3

In [ ]:
models = []
for _ in range(n_members):
	model = build_model()
	models.append(model)

In [ ]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32, validation_split=0.2)

In [ ]:
y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_).squeeze()
predicted = (y_>0.5).astype(np.int)

import scipy
outcomes, _ = scipy.stats.mode(predicted)
print(outcomes[:10])
outcomes = outcomes.squeeze()


from sklearn.metrics import accuracy_score

acc = accuracy_score(test_y, outcomes)
print("acc = ", acc)